## CryptoBot - data understanding historical

#### 1. Install Libraries

2.1 pip install pandas </br>
2.2 pip install python-binance </br>
2.3 pip install python-dotenv

#### 3. Import Libraries & API Keys

In [1]:
import pandas as pd
from binance import BinanceSocketManager
from dotenv import load_dotenv
import os
from binance.client import Client

In [3]:
load_dotenv()
binance_api_key = os.getenv("BINANCE_API_KEY")
binance_api_secret = os.getenv("BINANCE_API_SECRET")
client = Client(binance_api_key, binance_api_secret, testnet=True)

## Historical Data (Market Data Endpoint)
###### https://binance-docs.github.io/apidocs/spot/en/#market-data-endpoints

### 1.1 Klines based

The data displayed in the Klines (Candlesticks) is based on the completed transactions between buyers and sellers on the Binance exchange.

In [4]:
# Get klines from market Data Endpoint
def get_hist_klines_data(symbol, interval, start_str):
    candles = client.get_historical_klines(
        symbol=symbol,
        interval=interval,
        start_str=start_str
    )
    return candles

# Build dataframe
def build_klines_data_frame(candles):
    columns = ['Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time',
            'Quote asset volume', 'Number of trades', 'Taker buy base asset volume',
            'Taker buy quote asset volume', 'Ignore']
    
    df = pd.DataFrame(candles, columns=columns)
    df['Open time'] = pd.to_datetime(df['Open time'], unit='ms')
    df['Close time'] = pd.to_datetime(df['Close time'], unit='ms')
    df = df[['Open time', 'Open', 'High', 'Low', 'Close', 'Volume']]

    return df.tail()

### 1.2 Trade based

These data reflect the completed transactions where buyers and sellers have executed their orders on the platform.

In [5]:
# Get trades from Market Data Endpoint
def get_hist_trad_data(symbol, limit):
    
    historical_trades = client.get_historical_trades(symbol=symbol, limit=limit)

    return historical_trades

# Build dataframe
def build_trad_data_frame(historical_trades):

    df = pd.DataFrame(historical_trades)
    df['time'] = pd.to_datetime(df['time'], unit='ms')

    return df.head()

In [6]:
# print to json file
def print_to_json_file(filename,df):
    
    #filename = symbol +'_historical_klines.json'
    json_str = df.to_json(orient='records')  

    with open(filename, "w") as file:
        file.write(json_str)
    print("Text file written!", filename)

### run script klines

In [34]:
# Select parameters
symbol_array = ["BTCUSDT", "ETHBTC"] 
interval = Client.KLINE_INTERVAL_1HOUR 
start_str="41 days ago UTC"

for symbol in symbol_array:

    filename = symbol +'_hist_klines.json'

    candles = get_hist_klines_data(symbol, interval, start_str)
    df = build_klines_data_frame(candles)
    print_to_json_file(filename, df)

Text file written! BTCUSDT_hist_klines.json
Text file written! ETHBTC_hist_klines.json


### run script trades

In [ ]:
# Select parameters
symbol_array = ["BTCUSDT", "ETHBTC"] 
limit=1000

for symbol in symbol_array:

    filename = symbol +'_hist_trades.json'

    trades = get_hist_trad_data(symbol, limit)
    df = build_trad_data_frame(trades)
    print_to_json_file(filename, df)


Text file written! BTCUSDT_hist_trades.json
Text file written! ETHBTC_hist_trades.json


## Streaming Data (Websocket Market Endpoint)
###### https://binance-docs.github.io/apidocs/spot/en/#websocket-market-streams

In [35]:
import json
import websocket
import pandas as pd
import threading
import os

In [42]:
df = pd.DataFrame()

def on_message(ws, message):
    msg = json.loads(message)
    d = [(msg['T'],msg['p'])]
    global df
    df = pd.concat([df, pd.DataFrame.from_records(d)])
    #print(message)

def on_error(ws, error):
    print(error)

def on_close(ws, close_status_code, close_msg):
    print('### closed ###')
    df.columns = ['time', 'price']
    df['time'] = pd.to_datetime(df['time'], unit='ms')
    #print(df)
    print_to_json_file(filename,df)
    
def on_open(ws):
    print("Opened connection")
    # Start a timer to close the WebSocket after 40 seconds
    def stop_stream():
        print("Closing WebSocket after 40 seconds...")
        ws.close()
    
    timer = threading.Timer(5, stop_stream)
    timer.start()

def print_to_json_file(filename,df):
    
    json_str = df.to_json(orient='records')  

    with open(filename, "w") as file:
        file.write(json_str)
    print("Text file written!", filename)

In [41]:
symbol_array = ['btcusdt','ethbtc']
path = "streaming_data"
os.mkdir(path)

for symbol in symbol_array:
    socket = f'wss://stream.binance.com:9443/ws/{symbol}@trade'
    filename = path + '/' + symbol.upper() +'_hist_trades.json'
    ws = websocket.WebSocketApp(socket, on_open=on_open, on_message=on_message, on_error=on_error,on_close=on_close)
    ws.run_forever()

Opened connection
### closed ###
Text file written! streaming_data/BTCUSDT_hist_trades.json
Opened connection
### closed ###
Text file written! streaming_data/ETHBTC_hist_trades.json


In [82]:
symbol = 'btcusdt'
socket = f'wss://stream.binance.com:9443/ws/{symbol}@kline_1m'

In [83]:
df = pd.DataFrame()

def on_message(ws, message):
    print(message)

def on_error(ws, error):
    print(error)

def on_close(ws, close_status_code, close_msg):
    print('### closed ###')

def on_open(ws):
    print("Opened connection")

ws = websocket.WebSocketApp(socket, on_open=on_open, on_message=on_message, on_error=on_error, on_close=on_close)

In [84]:
ws.run_forever()

Opened connection
{"e":"kline","E":1734100154017,"s":"BTCUSDT","k":{"t":1734100140000,"T":1734100199999,"s":"BTCUSDT","i":"1m","f":4266205701,"L":4266207536,"o":"100371.25000000","c":"100414.70000000","h":"100414.70000000","l":"100371.25000000","v":"1.64591000","n":1836,"x":false,"q":"165222.10612060","V":"1.58488000","Q":"159095.49878490","B":"0"}}
{"e":"kline","E":1734100156014,"s":"BTCUSDT","k":{"t":1734100140000,"T":1734100199999,"s":"BTCUSDT","i":"1m","f":4266205701,"L":4266207546,"o":"100371.25000000","c":"100414.70000000","h":"100414.70000000","l":"100371.25000000","v":"1.65009000","n":1846,"x":false,"q":"165641.83954340","V":"1.58674000","Q":"159282.27012690","B":"0"}}

### closed ###


True

In [ ]:
t = []

def on_message(ws, message):
    msg = json.loads(message)
    bar = msg['k']
    if bar['x'] == True:
        t.append(bar['t'])

### Transaction

#### 6. Order Book

In [39]:
market_depth = client.get_order_book(symbol='BTCUSDT')
bids = pd.DataFrame(market_depth['bids'])
bids.columns = ['price','bids']
asks = pd.DataFrame(market_depth['asks'])
asks.columns = ['price','asks']
df = pd.concat([bids,asks]).fillna(0)
df.head()

,price,bids,asks
0,96786.29000000,0.00100000,0
1,96596.69000000,0.00001000,0
2,96539.63000000,0.00312000,0
3,96538.15000000,0.00130000,0
4,96536.56000000,0.00513000,0
